# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Introduction-to" data-toc-modified-id="Introduction-to-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction to</a></div><div class="lev2 toc-item"><a href="#What's-Julia?" data-toc-modified-id="What's-Julia?-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>What's Julia?</a></div><div class="lev2 toc-item"><a href="#R-is-great,-but..." data-toc-modified-id="R-is-great,-but...-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>R is great, but...</a></div><div class="lev2 toc-item"><a href="#Gibbs-sampler-example-by-Doug-Bates" data-toc-modified-id="Gibbs-sampler-example-by-Doug-Bates-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Gibbs sampler example by Doug Bates</a></div><div class="lev2 toc-item"><a href="#Julia-REPL-(Read-Evaluation-Print-Loop)" data-toc-modified-id="Julia-REPL-(Read-Evaluation-Print-Loop)-14"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Julia REPL (Read-Evaluation-Print-Loop)</a></div><div class="lev2 toc-item"><a href="#Which-Editor/IDE?" data-toc-modified-id="Which-Editor/IDE?-15"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Which Editor/IDE?</a></div><div class="lev2 toc-item"><a href="#Julia-package-system" data-toc-modified-id="Julia-package-system-16"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Julia package system</a></div><div class="lev2 toc-item"><a href="#Calling-R-from-Julia" data-toc-modified-id="Calling-R-from-Julia-17"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Calling R from Julia</a></div><div class="lev2 toc-item"><a href="#Some-basic-Julia-code" data-toc-modified-id="Some-basic-Julia-code-18"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>Some basic Julia code</a></div><div class="lev2 toc-item"><a href="#Random-numbers" data-toc-modified-id="Random-numbers-19"><span class="toc-item-num">1.9&nbsp;&nbsp;</span>Random numbers</a></div><div class="lev2 toc-item"><a href="#Timing-and-benchmark" data-toc-modified-id="Timing-and-benchmark-110"><span class="toc-item-num">1.10&nbsp;&nbsp;</span>Timing and benchmark</a></div><div class="lev3 toc-item"><a href="#Julia" data-toc-modified-id="Julia-1101"><span class="toc-item-num">1.10.1&nbsp;&nbsp;</span>Julia</a></div><div class="lev3 toc-item"><a href="#R" data-toc-modified-id="R-1102"><span class="toc-item-num">1.10.2&nbsp;&nbsp;</span>R</a></div><div class="lev3 toc-item"><a href="#C" data-toc-modified-id="C-1103"><span class="toc-item-num">1.10.3&nbsp;&nbsp;</span>C</a></div><div class="lev3 toc-item"><a href="#Python,-builtin-sum" data-toc-modified-id="Python,-builtin-sum-1104"><span class="toc-item-num">1.10.4&nbsp;&nbsp;</span>Python, builtin <code>sum</code></a></div><div class="lev3 toc-item"><a href="#Python,-numpy" data-toc-modified-id="Python,-numpy-1105"><span class="toc-item-num">1.10.5&nbsp;&nbsp;</span>Python, numpy</a></div><div class="lev3 toc-item"><a href="#Python,-handwritten" data-toc-modified-id="Python,-handwritten-1106"><span class="toc-item-num">1.10.6&nbsp;&nbsp;</span>Python, handwritten</a></div>

# Introduction to

<img src="./julia_logo.png" alt="Julia toolchain" style="width: 400px;"/>

In [ ]:
# machine information
versioninfo()

## What's Julia?

> Julia is a high-level, high-performance dynamic programming language for technical computing, with syntax that is familiar to users of other technical computing environments

* Started in 2009. First public release in 2012. 
  - Creators: Jeff Bezanson, Alan Edelman, Stefan Karpinski, Viral Shah
  - Current release v0.6.4
  - v1.0 is staged to release in 2018

* Aim to solve the notorious **two language problem**:
    - Prototype code goes into high-level languages like R/Python, production code goes into low-level language like C/C++
> Walks like Python. Runs like C.

* Write high-level, abstract code that closely resembles mathematical formulas
    - yet produces fast, low-level machine code that has traditionally only been generated by static languages.
    
<img src="./julia_vs_otherlang.png" align="center" width="600"/>    

* Julia is more than just "Fast R" or "Fast Matlab"
    - Performance comes from features that work well together.  
    - You can't just take the magic dust that makes Julia fast and sprinkle it on [language of choice]

## R is great, but...

* It's not meant for high performance computing
  - http://adv-r.had.co.nz/Performance.html
    - Section on performance starts with "Why is R slow?" 

* Deficiencies in the core language 
  - Many fixed with packages (`devtools`, `roxygen2`, `Matrix`, `tidyverse`)
  - Others harder to fix (R uses an old version of BLAS)
  - Some impossible to fix (clunky syntax, poor design choices)

* Only 6 active developers left (out of 20 R-Core members)
  - JuliaLang organization has 74 members, with 567 total contributors (as of 3/3/17)
    - https://github.com/JuliaLang/julia/graphs/contributors

* Doug Bates (member of R-Core, `Matrix` and `lme4`)
    - Getting Doug on board was a big win for statistics with Julia, as he brought a lot of knowledge about the history of R development and design choices
    
    > As some of you may know, I have had a (rather late) mid-life crisis and run off with another language called Julia.   
    >
    > -- <cite>Doug Bates (on the `knitr` Google Group)</cite>

## Gibbs sampler example by Doug Bates

* An example from Dr. Doug Bates's slides [Julia for R Programmers](http://www.stat.wisc.edu/~bates/JuliaForRProgrammers.pdf).

* The task is to create a Gibbs sampler for the density  
$$
f(x, y) = k x^2 exp(- x y^2 - y^2 + 2y - 4x), x > 0
$$
using the conditional distributions
$$
\begin{eqnarray*}
  X | Y &\sim& \Gamma \left( 3, \frac{1}{y^2 + 4} \right) \\
  Y | X &\sim& N \left(\frac{1}{1+x}, \frac{1}{2(1+x)} \right).
\end{eqnarray*}
$$

* **R solution**. The `RCall.jl` package allows us to execute R code without leaving the `Julia` environment. We first define an R function `Rgibbs()`.

In [ ]:
# Pkg.add("RCall")
using RCall

R"""
library(Matrix)
Rgibbs <- function(N, thin) {
  mat <- matrix(0, nrow=N, ncol=2)
  x <- y <- 0
  for (i in 1:N) {
    for (j in 1:thin) {
      x <- rgamma(1, 3, y * y + 4) # 3rd arg is rate
      y <- rnorm(1, 1 / (x + 1), 1 / sqrt(2 * (x + 1)))
    }
    mat[i,] <- c(x, y)
  }
  mat
}
"""

and then generate a sample of size 10,000 with a thinning of 500

In [ ]:
R"""
Rgibbs(100, 5); # warm-up
system.time(Rgibbs(10000, 500))
"""

* **Julia solution**. This is a Julia function for the same Gibbs sampler:

In [ ]:
# Pkg.add("Distributions")
using Distributions

function jgibbs(N, thin)
    mat = zeros(N, 2)
    x = y = 0.0
    for i in 1:N
        for j in 1:thin
            x = rand(Gamma(3, 1 / (y * y + 4)))
            y = rand(Normal(1 / (x + 1), 1 / sqrt(2(x + 1))))
        end
        mat[i, 1] = x
        mat[i, 2] = y
    end
    mat
end

Generate the same sample. How long does it take?

In [ ]:
jgibbs(100, 5); # warm-up
@elapsed jgibbs(10000, 500)

We see ~50 fold speed up of `Julia` over `R` on this example, **without extra coding effort**!

## Julia REPL (Read-Evaluation-Print-Loop)

The `Julia` REPL, or `Julia` shell, has four main modes.

0. **Default mode** is the Julian prompt `julia>`. Type backspace in other modes to enter default mode.    

0. **Help mode** `help?>`. Type `?` to enter help mode. `?search_term` does a fuzzy search for `search_term`. 

0. **Shell mode** `shell>`. Type `;` to enter shell mode.  

0. **Search mode** `(reverse-i-search)`. Press `ctrl+R` to enter search model. 

0. With `RCall.jl` package installed, we can enter the **R mode** by typing `$` (shift+4) at Julia REPL.

Some survival commands in Julia REPL:  
0. `quit()` or `Ctrl+D`: exit Julia.

0. `Ctrl+C`: interrupt execution.

0. `Ctrl+L`: clear screen.

0. `whos()`: list all variables in current workspace.

0. `workspace()`: clear all variables and reset session.

0. Append `;` (semi-colon) to suppress displaying output from a command.

0. `include("filename.jl")` to source a Julia code file.

0. One very useful command to display the machine information is `versioninfo()`. `versioninfo(true)` gives detailed system information.

In [ ]:
versioninfo()

## Which Editor/IDE?

* I highly recommend  
    * Editor [Visual Studio Code](https://code.visualstudio.com) with the Julia extension installed, or  
    * Editor [Atom](https://atom.io) with packages `julia-client`, `language-julia`, and `latex-completions` installed. 

* If you want RStudio- or Matlab-like IDE, you can try 
    * [JupyterLab](https://github.com/jupyterlab/jupyterlab), or 
    * install the `uber-juno` package in Atom. Follow instructions at [https://github.com/JunoLab/uber-juno/blob/master/setup.md](https://github.com/JunoLab/uber-juno/blob/master/setup.md).

* [**JuliaPro**](https://juliacomputing.com/products/juliapro.html) bundles Julia, Atom, Juno, and many commonly used packages. It has an option to use Intel MKL (Mathematical Kernel Library) instead of OpenBLAS for linear algebra.

## Julia package system

<img src="http://pkg.julialang.org/img/allver.svg" width="500" align="center"/>

* Each Julia package is a Git repository. Each Julia package name ends with `.jl`. E.g., `Distributions.jl` package lives at <https://github.com/JuliaStats/Distributions.jl>.   
Google search with `PackageName.jl` usually leads to the package on github.com. 

* The package ecosystem is rapidly maturing; a complete list of **registered** packages (which are required to have a certain level of testing and documentation) is at [http://pkg.julialang.org/](http://pkg.julialang.org/).

* For example, the package called `Distributions.jl` is added with
```julia
Pkg.add("Distributions")   # no .jl 
```
and "removed" (although not completely deleted) with
```julia
Pkg.rm("Distributions")
```
* The package manager provides a dependency solver that determines which packages are actually required to be installed.

* **Non-registered** packages are added by cloning the relevant Git repository. E.g.,
```julia
Pkg.clone("git@github.com:OpenMendel/SnpArrays.jl.git")
```

* A package needs only be added once, at which point it is downloaded into your local `.julia/vx.x` directory in your home directory. 

* Directory of a specific package can be queried by `Pkg.dir()`:

In [ ]:
Pkg.dir("Distributions")

* If you start having problems with packages that seem to be unsolvable, you can try just deleting your .julia directory and reinstalling all your packages. 

* For **JuliaPro**, the packages are downloaed to directory `/Applications/JuliaPro-0.6.2.2.app/Contents/Resources/pkgs-0.6.2.2/v0.6/`.

* Periodically, one should run `Pkg.update()`, which checks for, downloads and installs updated versions of all the packages you currently have installed.

* `Pkg.status()` lists the status of all installed packages.

* Using functions in package.
```julia
using Distributions
```
This pulls all of the *exported* functions in the module into your local namespace, as you can check using the `whos()` command. An alternative is
```julia
import Distributions
```
Now, the functions from the Distributions package are available only using 
```julia
Distributions.<FUNNAME>
```
All functions, not only exported functions, are always available like this.

## Calling R from Julia

* The [`RCall.jl`](https://github.com/JuliaInterop/RCall.jl) package allows you to embed R code inside of Julia. Internally it calls the R installed on the same computer.

* For other languages, check [JuliaInterop](https://github.com/JuliaInterop): `PyCall`, `MATLAB`, `JavaCall`, `CxxWrap`, ... 

In [ ]:
# Pkg.add("RCall")
using RCall

# Julia vector x is interpolated into R workspace
x = randn(1000)
R"""
hist($x, main="I'm plotting a Julia vector")
"""

In [ ]:
R"""
library(ggplot2)
qplot($x)
"""

In [ ]:
# wrapper of R variable(s)
x = R"""
rnorm(10)
"""

In [ ]:
# convert it into Julia variable
y = collect(x)

* Access Julia variables in R REPL mode:
```julia
julia> x = rand(5) # Julia variable
R> y <- $x
```

* Pass Julia expression in R REPL mode:
```julia
R> y <- $(rand(5))
```

* Put Julia variable into R environment:
```julia
julia> @rput x
R> x
```

* Get R variable into Julia environment:
```julia
R> r <- 2
Julia> @rget r
```

* If you want to call Julia within R, check out the [`XRJulia`](https://cran.r-project.org/web/packages/XRJulia/) package by John Chambers.

## Some basic Julia code

In [ ]:
y = 1
typeof(y) # same as int in R

In [ ]:
y = 1.0
typeof(y) # same as double in R

In [ ]:
# Greek letters:  `\pi<tab>`
π

In [ ]:
# Greek letters:  `\theta<tab>`
θ = y + π

In [ ]:
# emoji! `\:kissing_cat:<tab>`
😽 = 5.0

In [ ]:
# wear a hat: `\alpha<tab>\hat<tab>`
α̂ = π

In [ ]:
# vector of Float64 0s
x = zeros(5)

In [ ]:
# vector of Int64 0s
x = zeros(Int, 5)

In [ ]:
# matrix of Float64 0s
x = zeros(5, 3)

In [ ]:
# matrix of Float64 1s
x = ones(5, 3)

In [ ]:
# define array without initialization
x = Array{Float64}(5, 3)

In [ ]:
# fill an array by 0s
fill!(x, 0)

In [ ]:
# initialize an array to be 2.5
fill(2.5, (5, 3))

In [ ]:
a = 3//5

In [ ]:
typeof(a)

In [ ]:
b = 3//7
a + b

## Random numbers

In [ ]:
# set random seed
srand(2018)

In [ ]:
# uniform [0, 1) random numbers
x = rand(5, 3)

In [ ]:
# uniform random numbers (in single precision)
x = rand(Float32, 5, 3)

In [ ]:
# random numbers from {1,...,5}
x = rand(1:5, 5, 3)

In [ ]:
# standard normal random numbers
x = randn(5, 3)

To generate random samples from a specific distribution, we use the Distributions.jl package.

In [ ]:
using Distributions

# iid Exp(1)
rand(Exponential(1), 5, 3)

## Timing and benchmark

`@time`, `@elapsed`, `@allocated` macros:

In [ ]:
srand(123) # seed
x = randn(10^7)
@time sum(x) # first run includes compilation time
@time sum(x) # no compilation time after first run

In [ ]:
@elapsed sum(x)

In [ ]:
@allocated sum(x)

###  Julia

Use `BenchmarkTools.jl` for more robust benchmarking. Analog to `microbenchmark` package in R.

In [ ]:
using BenchmarkTools

@benchmark sum($x)

### R

In [ ]:
using RCall

R"""
library(microbenchmark)
microbenchmark(sum($x))
"""

Why is R ~1.5-2 fold slower than Julia?

### C

In [ ]:
C_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
"""

const Clib = tempname()   # make a temporary file


# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):

open(`gcc -std=c99 -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

In [ ]:
@benchmark c_sum($x)

### Python, builtin `sum`

In [ ]:
using PyCall
# get the Python built-in "sum" function:
pysum = pybuiltin("sum")
@benchmark $pysum($x)

### Python, numpy

In [ ]:
using Conda

numpy_sum = pyimport("numpy")["sum"]

@benchmark $numpy_sum($x)

### Python, handwritten

In [ ]:
py"""
def py_sum(A):
    s = 0.0
    for a in A:
        s += a
    return s
"""

sum_py = py"py_sum"

@benchmark $sum_py($x)